In [1]:
# This notebook take UVES images numpy arrays and compute the resnet50 descriptors
# After that it save the descriptors in the descriptor blob in azure
from azure.storage.blob import BlockBlobService, PublicAccess

In [2]:
# Create the BlockBlockService that is used to call the Blob service 
# for the storage account
AccountName = 'uvesdata'
AccountKey = 'UU/60Su8kB++k39FqnENlVgNM2TenRBNso80EKfrG2hqJTz0ah22Q6T1bo+BYYtIw7md/zjwIjr3kg0jvfMo/w=='
block_blob_service = BlockBlobService(account_name=AccountName, account_key=AccountKey)

# Create a container called 'descriptor' (it is already there).
ContNameDesc = 'descriptorcorrupt'
cont_name_desc = ContNameDesc
#block_blob_service.create_container(container_name)

# Set the permission so the blobs are public.
block_blob_service.set_container_acl(cont_name_desc, public_access=PublicAccess.Container)

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

%matplotlib inline

In [4]:
# Create a list "filelist" with the blob content
# inside the "Azure:container/folder" location 
def BlobList(container, folder, filelist, verbose=False):
    
    gen = block_blob_service.list_blobs(container, prefix=folder)
    
    for blob in gen:
        file = str(blob.name).replace(folder,'')
        filelist.append(file)
        if verbose == True:
            print("\t Blob name: " + blob.name)
        
    return filelist

# Download a file "blobfile" from "container" and save it 
# in the file "locfile"
def DownBlob(container, blobfile, locfile, verbose=False):
    
    if verbose == True:
        print('Downloading ' + blobfile + ' to ' + locfile)
    
    block_blob_service.get_blob_to_path(container, blobfile, locfile)

In [6]:
# List of numpy arrays to process
# * bias_red:
#   - UVES_RED_BIAS (ext: 1,2)
# * bias_blue:
#   - UVES_BLUE_BIAS (ext: 0)
# * blue_arc_flat:
#   - UVES_BLUE_WAVE (ext: 1,2)
#   - UVES_DIC1B_FLAT (ext: 0)
#   - UVES_DIC1B_DFLAT (ext: 0)
# * red_arc_flat:
#   - UVES_RED_WAVE (ext: 1,2)
#   - UVES_DIC1R_FLAT (ext: 1,2)
# The following lines produce the resnet50 descriptors for the images inside
# fits_folder and the corresponding extentsion 

# Define the subdirs to be created in /descriptor based on the image type and ext
#desc_blob_sub_dir = ['UVES_BLUE_BIAS','UVES_RED_BIAS','UVES_BLUE_WAVE','UVES_DIC1B_FLAT','UVES_DIC1B_DFLAT','UVES_RED_WAVE','UVES_DIC1R_FLAT']
# Test
desc_blob_sub_dir = 'UVES_BLUE_BIAS'

path_loc = './data'
    
if desc_blob_sub_dir == 'UVES_BLUE_BIAS' or desc_blob_sub_dir == 'UVES_DIC1B_FLAT' or desc_blob_sub_dir == 'UVES_DIC1B_DFLAT':
    Exten = [0]
elif desc_blob_sub_dir == 'UVES_RED_BIAS' or desc_blob_sub_dir == 'UVES_BLUE_WAVE' or desc_blob_sub_dir == 'UVES_RED_WAVE' or desc_blob_sub_dir == 'UVES_DIC1R_FLAT':
    Exten = [1,2]
    
# Loop for images extensions
for ext in Exten:
    print('\n')
    print('For Extension ' + str(ext))
    desc_files_list = []
    extension = 'ext'+str(ext)
    # List the images-extension data
    desc_folder_rem = os.path.join('numpy/method2', desc_blob_sub_dir)
    desc_folder_rem = os.path.join(desc_folder_rem, extension)
    BlobList(cont_name_desc, desc_folder_rem, desc_files_list)
    print('Uploaded descriptors: ',len(desc_files_list))
    # For random selection
    if len(desc_files_list)>0:
        desc_to_check = random.sample(desc_files_list, 1)
        desc_to_check = str(desc_to_check[0])
    
        desc_file_rem = desc_folder_rem + desc_to_check
        loc_file = path_loc + desc_to_check
        print('Checking descriptor ' + desc_file_rem)
    
        DownBlob(cont_name_desc, desc_file_rem, loc_file, False)

        desc = np.load(loc_file)
        np_desc = np.array(desc)
        print('Descriptor shape: ', np_desc.shape)
        print('Descriptor array: ', np_desc)
    elif len(desc_files_list)==0:
        print('No descriptors in ' + desc_folder_rem)



For Extension 0
Uploaded descriptors:  100
Checking descriptor numpy/method2/UVES_BLUE_BIAS/ext0/UVES.2017-04-06T10:46:29.449C297_desc.npy
Descriptor shape:  (1, 2048)
Descriptor array:  [[0.00741417 0.00418247 0.00958695 ... 0.03276253 0.00384135 0.        ]]
